In [1]:
import cv2

In [2]:
import os
import numpy as np
from scipy import ndimage, misc
from sklearn.utils import shuffle
from sklearn.cross_validation import StratifiedKFold
from sklearn import datasets, svm, metrics

import skimage
from skimage.feature import local_binary_pattern
from skimage import data

directory = './Images/'

def load(seed=123):
    X = []
    y = []
    for i in range(5):
        cur_direct = directory + 'Cluster ' + str(i + 1) + '/'
        for jpg in os.listdir(cur_direct):
            # Flatten for simpler models, don't want to do this for CNN
#             X.append(misc.imresize(ndimage.imread(cur_direct + jpg, flatten=True), 0.2))
#             X.append((ndimage.imread(cur_direct + jpg, flatten=False)/255.))
            X.append(skimage.transform.resize(data.imread(cur_direct + jpg,
                                                          flatten=True,
                                                          as_grey=True), 
                                              (256, 256)))
            y.append(i+1)
    
    X = np.asarray(X)
    y = np.asarray(y)
  
    X, y = shuffle(X, y, random_state=seed)  # shuffle train data
    
    X = X.astype('float32')
    y = y.astype('float32')
    return (X, y)
#     return (X, y)

X, y = load()
print(X.shape)
print(y.shape)

(347, 256, 256)
(347,)


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

radius = 6
n_points = 4 * radius
METHOD = 'uniform'

X_ls = []

for x in X:
    X_ls.append(np.asarray(local_binary_pattern(x, n_points, radius, METHOD)).ravel())

X_ls = np.asarray(X_ls)
print(len(X_ls))
print(type(X_ls[0]))
print(X_ls[0].shape)

347
<type 'numpy.ndarray'>
(65536,)


In [ ]:
# from skimage.transform import rotate
# from skimage.color import label2rgb

# # settings for LBP
# radius = 3
# n_points = 8 * radius

# skf = StratifiedKFold(y, n_folds=3)

# for train, test in skf:
    

In [4]:
skf = StratifiedKFold(y, n_folds=3)

for train, test in skf:
    print(X_ls.shape)
    print(y.shape)
    print(X_ls[train].shape)
    classifier = svm.SVC(kernel='poly', class_weight='balanced')
    classifier.fit(X_ls[train], y[train])

    # Now predict the value of the digit on the second half:
    expected = y[test]
    predicted = classifier.predict(X_ls[test])

    print("Classification report for classifier %s:\n%s\n"
          % (classifier, metrics.classification_report(expected, predicted)))
    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

(347, 65536)
(347,)
(229, 65536)
Classification report for classifier SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
             precision    recall  f1-score   support

        1.0       0.00      0.00      0.00        17
        2.0       0.48      1.00      0.65        36
        3.0       0.00      0.00      0.00        12
        4.0       0.72      0.84      0.77        37
        5.0       0.00      0.00      0.00        16

avg / total       0.37      0.57      0.44       118


Confusion matrix:
[[ 0  5  0 12  0]
 [ 0 36  0  0  0]
 [ 0 12  0  0  0]
 [ 0  6  0 31  0]
 [ 0 16  0  0  0]]
(347, 65536)
(347,)
(231, 65536)
Classification report for classifier SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='poly',
  max_iter

/home/brett/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
skf = StratifiedKFold(y, n_folds=3)

for train, test in skf:
    classifier = RandomForestClassifier(n_estimators=1000, class_weight="balanced")
    classifier.fit(X_ls[train], y[train])

    print(y[train])
    print(y[test])

    # Now predict the value of the digit on the second half:
    expected = y[test]
    predicted = classifier.predict(X_ls[test])

    print("Classification report for classifier %s:\n%s\n"
          % (classifier, metrics.classification_report(expected, predicted)))
    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
skf = StratifiedKFold(y, n_folds=3)

for train, test in skf:
    classifier = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    classifier.fit(X_ls[train], y[train])

    print(y[train])
    print(y[test])

    # Now predict the value of the digit on the second half:
    expected = y[test]
    predicted = classifier.predict(X_ls[test])

    print("Classification report for classifier %s:\n%s\n"
          % (classifier, metrics.classification_report(expected, predicted)))
    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
    
